# [예측모델 - Solution] 

## - 구간평균법 
## - 단순지수평활법 
## - 이중지수평활법 

### 1. 모듈 불러오기

In [ ]:
import os

import pandas as pd
import pandas_datareader.data as pdr

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

from sklearn.datasets import fetch_openml
import statsmodels.api as sm

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-whitegrid')

import seaborn as sns
#sns.set_style("white")

import itertools
%matplotlib inline

### 2. 데이터 불러오기

In [ ]:
# Google trend에서 2012년부터 2022년까지 10년간 '데이터분석 ' 키워드 관심도 변화량
# https://trends.google.com/trends/explore?date=2012-01-01%202022-01-01&geo=KR&q=%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D
data = pd.read_csv('./googletrend_keyword.csv')
data

#### 2.1 Data Preprocessing

In [ ]:
# change column name
data = data.rename(columns={'카테고리: 모든 카테고리': 'data_analysis(korea)'})
data = data[1:]

In [ ]:
# data.info()

In [ ]:
# change the column type to numeric
data = data.astype(np.int64(data['data_analysis(korea)']))

In [ ]:
# change the index type to datetime
data.index = pd.to_datetime(data.index)
data.index

In [ ]:
'''
데이터 시각화를 통해 전체 개요 확인
'''
data.plot(figsize=(12,4)) # color='green', linestyle='--', linewidth=1

plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.legend('')

plt.title("'Data analysis' keyword search amount from Google trend \n", fontsize=15)
plt.xlabel('\n Year', fontsize=13)
plt.ylabel('keyword search amount \n', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Seasonal decomposition plot: Seasonal decomposition using moving averages.
# https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

# Observed: observed data
# Trend: The estimated trend component
# Seasonal: The estimated seasonal component
# resid: The estimated residuals

decompostion = sm.tsa.seasonal_decompose(data['data_analysis(korea)'],  model='additive')

fig = decompostion.plot()
fig.set_size_inches(10,10)
plt.show()

In [ ]:
'''
Train Test Split
'''
train = data[:'2019-12']
test  = data['2020-01':]

### 3. Moving Average (구간평균법)

In [ ]:
'''
Version 1
'''
def Moving_Average(x, N):
    x = x.values.flatten()
    
    pred = np.convolve(x, np.ones(N) / float(N), 'valid')
    pred = np.concatenate((np.zeros(N-1), pred), axis=0)
    pred[:N-1] = np.nan
    return pred

MA_train_pred = pd.DataFrame(Moving_Average(train, 5), index=train.index, columns=['MA_5'])
MA_test_pred = pd.DataFrame(np.array([MA_train_pred.iloc[-1]]*len(test)), index=test.index, columns=['MA_5'])

print('Moving Average Train results')
print(MA_train_pred)
print('-'*30)
print('Moving Average Test results')
print(MA_test_pred)

In [ ]:
'''
Version 2
'''
MA_train_pred = train.rolling(5).mean() # Option: min_periods=1
MA_train_pred.columns = ['MA_5']

MA_test_pred = pd.DataFrame(np.array([MA_train_pred.iloc[-1]]*len(test)), index=test.index, columns=['MA_5'])
prediction = pd.concat([MA_train_pred, MA_test_pred], axis=0)

print('Moving Average Train Results')
print(MA_train_pred)
print('-'*30)
print('Moving Average Test Results')
print(MA_test_pred)

In [ ]:
'''
Visualization 
'''
fig, ax = plt.subplots(figsize=(12,4))
data.plot(ax=ax)
prediction.plot(ax=ax, label = 'Prediction (N=5)') #ax 로 하나의 plot에 표현
ax.vlines(test.index[0], 0, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'Prediction (N=5)', 'Start of Forecast'], loc='upper left')
plt.title('Moving Average Results (Train and Test)')
plt.tight_layout()
plt.show()

# Only Test
fig, ax = plt.subplots(figsize=(12,4))
data.plot(ax=ax)
MA_test_pred.plot(ax=ax, label = 'Prediction (N=5)')
ax.vlines(test.index[0], 0, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'Prediction (N=5)', 'Start of Forecast'], loc='upper left')
plt.title('Moving Average Results (Only Test)')
plt.tight_layout()
plt.show()

##### 정량적 지표를 통한 모델 검증

###### Mean Squared Error (평균 제곱 오차) <br>
$\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}$   

In [ ]:
print(f'MSE: {np.round(mean_squared_error(test, MA_test_pred), 2)}')

######  Root Mean Squared Error (제곱근 평균 제곱 오차) <br>
$\sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}}$

In [ ]:
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,MA_test_pred)), 2)}')

##### Mean Absolute Error (평균 절대 오차) <br>
$\frac{1}{n} \sum_{i=1}^{n} |y_{i} - \hat{y}_{i}|$

In [ ]:
print(f'MAE: {np.round(mean_absolute_error(test, MA_test_pred), 2)}')

#####  R squared (결정계수 $({r})^{2}$) <br>
$\frac{SSR}{SST} = 1- \frac{SSR}{SST}%$

In [ ]:
print(f'R2 score: {np.round(r2_score(test, MA_test_pred), 2)}')

### 4. Exponential Smoothing (지수평활법)

#### 4.1 Simple Exponential Smoothing (단순지수평활법)

In [ ]:
L_0 = np.mean(train)  # initial value

SES_train = train.copy()
SES_train = pd.concat([pd.DataFrame(np.zeros(1), columns=['data_analysis(korea)']), SES_train])
SES_train['Level'] = np.nan
SES_train['Forecast'] = np.nan
SES_train.loc[0,'Level'] = L_0.values

SES_train

In [ ]:
'''
Version 1
'''
def Simple_Exponential_Smoothing(table, alpha):
    # table = SES_train.copy()
    # alpha = 0.9
    for i in range(len(table)):
        if i == 0: # Skip initialization point
            continue

        x = table.loc[list(table.index)[i],'data_analysis(korea)']
        L_prev = table.loc[list(table.index)[i-1],'Level']
        table.loc[list(table.index)[i],'Level'] = alpha*x + (1-alpha)*(L_prev)

        L_pred = table.iloc[-1,1]
        SES_test_pred = [L_pred]*len(test)
        SES_test_pred = pd.DataFrame(SES_test_pred, index=test.index, columns=[f'SES_{alpha}'])
        
    return table, SES_test_pred

In [ ]:
SES_train_pred, SES_test_pred = Simple_Exponential_Smoothing(SES_train, 0.9)

In [ ]:
print('Simple Exponential Smoothing Train Results')
print(SES_train_pred)
print('-'*30)
print('Simple Exponential Smoothing Test results')
print(SES_test_pred)

In [ ]:
'''
Version 2
'''

SES_train_pred_09 = pd.concat([pd.DataFrame([L_0], columns=['data_analysis(korea)']), train]).ewm(alpha=0.9, adjust=False).mean().iloc[1:] #mean?
SES_train_pred_09.columns = ['SES_09']
SES_test_pred_09 = pd.DataFrame(np.array([SES_train_pred_09.iloc[-1]]*len(test)),
                                index=test.index, columns=['SES_09'])
prediction_09 = pd.concat([SES_train_pred_09, SES_test_pred_09], axis=0)

SES_train_pred_05 = pd.concat([pd.DataFrame([L_0], columns=['data_analysis(korea)']), train]).ewm(alpha=0.5, adjust=False).mean().iloc[1:]
SES_train_pred_05.columns = ['SES_05']
SES_test_pred_05 = pd.DataFrame(np.array([SES_train_pred_05.iloc[-1]]*len(test)),
                                index=test.index, columns=['SES_05'])
prediction_05 = pd.concat([SES_train_pred_05, SES_test_pred_05], axis=0)

SES_train_pred_01 = pd.concat([pd.DataFrame([L_0], columns=['data_analysis(korea)']), train]).ewm(alpha=0.1, adjust=False).mean().iloc[1:]
SES_train_pred_01.columns = ['SES_01']
SES_test_pred_01 = pd.DataFrame(np.array([SES_train_pred_01.iloc[-1]]*len(test)),
                                index=test.index, columns=['SES_01'])
prediction_01 = pd.concat([SES_train_pred_01, SES_test_pred_01], axis=0)

In [ ]:
'''
Visualization 
'''
fig, ax = plt.subplots(figsize=(12,4))
data.plot(ax=ax)
prediction_09.plot(ax=ax, label = 'Prediction (alpha=0.9)')
prediction_05.plot(ax=ax, label = 'Prediction (alpha=0.5)')
prediction_01.plot(ax=ax, label = 'Prediction (alpha=0.1)')

ax.vlines(test.index[0], 0, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'Prediction (alpha=0.9)', 'Prediction (alpha=0.5)', 'Prediction (alpha=0.1)', 'Start of Forecast'], loc='upper left')
plt.suptitle('Exponential Moving Average Results')
plt.tight_layout()
plt.show()

# Only Test
fig, ax = plt.subplots(figsize=(12,4))
data['2020-01':'2022-01'].plot(ax=ax)
prediction_09['2020-01':'2022-01'].plot(ax=ax, label = 'Prediction (alpha=0.9)')
prediction_05['2020-01':'2022-01'].plot(ax=ax, label = 'Prediction (alpha=0.5)')
prediction_01['2020-01':'2022-01'].plot(ax=ax, label = 'Prediction (alpha=0.1)')

ax.vlines(test.index[0], 50, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'Prediction (alpha=0.9)', 'Prediction (alpha=0.5)', 'Prediction (alpha=0.1)', 'Start of Forecast'], loc='upper left')
plt.suptitle('Exponential Moving Average Results')
plt.tight_layout()
plt.show()

#### 4.2 Double Exponential Smoothing (이중지수평활법)

In [ ]:
'''
Searching Initialization Points
'''
train_reg = train.reset_index()
x = np.array(train_reg.index)
y = train_reg['data_analysis(korea)']

x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()

print(results.summary())
L_0, B_0 = results.params

In [ ]:
DES_train = train.copy()
DES_train = pd.concat([pd.DataFrame(np.zeros(1), columns=['data_analysis(korea)']), DES_train])
DES_train['Level'] = np.nan
DES_train['Trend'] = np.nan
DES_train['Forecast'] = np.nan

DES_train.loc[0,'Level'] = L_0
DES_train.loc[0,'Trend'] = B_0

DES_train

In [ ]:
def Double_Exponential_Smoothing(table, alpha, beta):
    # table = DES_train.copy()
    # alpha = 0.16
    # beta = 0.1
    for i in range(len(table)):
        if i == 0: # Skip initialization point
            continue

        x = table.loc[list(table.index)[i],'data_analysis(korea)']
        L_prev = table.loc[list(table.index)[i-1],'Level']
        T_prev = table.loc[list(table.index)[i-1],'Trend']
        
        table.loc[list(table.index)[i],'Level'] = alpha*x + (1-alpha)*(L_prev+T_prev)
        table.loc[list(table.index)[i],'Trend'] = beta*(table.loc[list(table.index)[i],'Level']-L_prev) + (1-beta)*(T_prev)

        L_Pred = table.iloc[-1,1]
        T_Pred = table.iloc[-1,2]
        
        DES_test_pred = L_Pred + range(len(test))*T_Pred
        DES_test_pred = pd.DataFrame(DES_test_pred, index = test.index, columns=['DES'])
        
    return table, DES_test_pred

In [ ]:
DES_train_pred, DES_test_pred = Double_Exponential_Smoothing(DES_train, 0.16, 0.1)

In [ ]:
print('Double Exponential Smoothing Train Results')
print(DES_train_pred)
print('-'*30)
print('Double Exponential Smoothing Test results')
print(DES_test_pred)

In [ ]:
'''
Visualization 
'''
# Train and Test
train_pred = pd.DataFrame(DES_train_pred.iloc[1:,1])
train_pred.columns = ['DES']
DES_pred = pd.concat([train_pred, DES_test_pred])

fig, ax = plt.subplots(figsize=(12,4))
data.plot(ax=ax)
DES_pred.plot(ax=ax, label = 'Prediction (alpha=0.16, beta=0.1)')
ax.vlines(test.index[0], 0, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'DEMA (alpha=0.16, beta=0.1)', 'Start of Forecast'], loc='upper left')
plt.suptitle('Double Exponential Moving Average Results')
plt.tight_layout()
plt.show()

In [ ]:
'''
Comparing
'''
fig, ax = plt.subplots(figsize=(12,4))
data['2020-01':].plot(ax=ax)
MA_test_pred['2020-01':].plot(ax=ax, label = 'Prediction (N=5)')
SES_test_pred_09['2020-01':].plot(ax=ax, label = 'Prediction (alpha=0.9)')
DES_test_pred['2020-01':].plot(ax=ax, label = 'Prediction (alpha=0.16, beta=0.1)')
ax.legend(['Raw Dataset', 'MA (N=5)', 'SES (alpha=0.9)', 'DES (alpha=0.16, beta=0.1)'], loc='upper left')
plt.suptitle('Double Exponential Moving Average Results')
plt.tight_layout()
plt.show()

#### 4.3 Holt-Winter's Exponential Smoothing (홀트-윈터 지수평활법)

##### 4.3.1 Additive Winter's method

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
HW_model = ExponentialSmoothing(train, trend='add', seasonal='add').fit(optimized=True)
HW_model.summary()

In [ ]:
HW_train_pred = HW_model.fittedvalues
HW_test_pred = HW_model.forecast(len(test))
HW_test_pred = pd.DataFrame(HW_test_pred, index=test.index, columns=['HW_add'])

print('Holt-Winter Exponential Smoothing Train Results')
print(HW_train_pred)
print('-'*30)
print('Holt-Winter Smoothing Test results')
print(HW_test_pred)

In [ ]:
'''
Visualization 
'''
# Train and Test
train_pred = pd.DataFrame(HW_train_pred, columns=['HW_add'])
HW_add_pred = pd.concat([train_pred, HW_test_pred])

fig, ax = plt.subplots(figsize=(12,4))
data.plot(ax=ax)
HW_add_pred.plot(ax=ax, label = 'Prediction')
ax.vlines(test.index[0], 0, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'HW', 'Start of Forecast'], loc='upper left')
plt.title('Holt-Winter Exponential Moving Average Results')
plt.tight_layout()
plt.show()

In [ ]:
'''
Comparing
'''
fig, ax = plt.subplots(figsize=(12,4))
data.plot(ax=ax)
MA_test_pred['2020-01':].plot(ax=ax, label = 'Prediction (N=5)')
SES_test_pred_09['2020-01':].plot(ax=ax, label = 'Prediction (alpha=0.9)')
DES_test_pred['2020-01':].plot(ax=ax, label = 'Prediction (alpha=0.16, beta=0.1)')
HW_test_pred['2020-01':].plot(ax=ax, label = 'Prediction (additive)')
ax.vlines(test.index[0], 0, 100, linestyle='--', color='r')
ax.legend(['Raw Dataset', 'MA (N=5)', 'SES (alpha=0.9)', 'DES (alpha=0.16, beta=0.1)', 'HW (Additive)'], loc='upper left')
plt.title('Double Exponential Moving Average Results')
plt.tight_layout()
plt.show()

In [ ]:
'''
Quantitative Evaluation
'''
print('-'*55)
print('Moving Average (N = 5)')
print(f'MSE: {np.round(mean_squared_error(test, MA_test_pred), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,MA_test_pred)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, MA_test_pred), 2)}')

print('-'*55)
print('Simple Exponential Smoothing (alpha = 0.9)')
print(f'MSE: {np.round(mean_squared_error(test, SES_test_pred_09), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,SES_test_pred_09)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, SES_test_pred_09), 2)}')

print('-'*55)
print('Double Exponential Smoothing (alpha = 0.16, beta = 0.1)')
print(f'MSE: {np.round(mean_squared_error(test, DES_test_pred), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,DES_test_pred)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, DES_test_pred), 2)}')

print('-'*55)
print('Additive Holt-Winter Exponential Smoothing')
print(f'MSE: {np.round(mean_squared_error(test, HW_test_pred), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,HW_test_pred)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, HW_test_pred), 2)}')

In [ ]:
test_predict_results = pd.concat([test, MA_test_pred, SES_test_pred_09, DES_test_pred, HW_test_pred], axis=1)
test_predict_results

# EOD